In [1]:
#Packages:
import pandas as pd
import numpy as np
import pyreadr as pyr
import sklearn as sk
import matplotlib.pyplot as plt
import scipy as sc
import os
import nltk
import spacy
from nltk.corpus import stopwords
from sklearn.impute import KNNImputer
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from scipy.stats import chi2
from sklearn.impute import KNNImputer
from sklearn.metrics import make_scorer
from numpy.random import normal
from numpy.random import seed
from sklearn.linear_model import Lasso
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from mlens.ensemble import SuperLearner
from sklearn.linear_model import LinearRegression

[MLENS] backend: threading


In [2]:
os.chdir("C:/Users/hp/OneDrive - Universidad de los Andes/Documentos/Docs/Universidad/2022-2/Big Data/Taller 3/Repo/PS3_BD-ML")
x_train = pd.read_csv("x_train.csv")
y_train = pd.read_csv("y_train.csv")
x_test = pd.read_csv("x_test.csv")

In [3]:
def losscore(y,ypred):
    penalty=np.abs(y-ypred)
    penalty=np.where(penalty<0.066, np.exp(penalty),penalty)
    penalty=np.where(penalty>0,np.exp(penalty),penalty)
    abs_exp=np.mean(penalty)
    return abs_exp

losscore2=make_scorer(losscore, greater_is_better=False)

In [4]:
models = []
	
models.append(xgb.XGBRegressor(n_estimators=10,eta=0.1, max_depth=1))
models.append(xgb.XGBRegressor(n_estimators=100,eta=0.1, max_depth=1))
models.append(xgb.XGBRegressor(n_estimators=500,eta=0.1, max_depth=1))
models.append(xgb.XGBRegressor(n_estimators=10,eta=0.1, max_depth=5))
models.append(xgb.XGBRegressor(n_estimators=100,eta=0.1, max_depth=5))
models.append(xgb.XGBRegressor(n_estimators=500,eta=0.1, max_depth=5))
models.append(xgb.XGBRegressor(n_estimators=10,eta=0.1, max_depth=10))
models.append(xgb.XGBRegressor(n_estimators=100,eta=0.1, max_depth=10))
models.append(xgb.XGBRegressor(n_estimators=500,eta=0.1, max_depth=10))

models.append(MLPRegressor(hidden_layer_sizes=(100,1), alpha=0.001, random_state=911))
models.append(MLPRegressor(hidden_layer_sizes=(250,5), alpha=0.001, random_state=911))
models.append(MLPRegressor(hidden_layer_sizes=(100,1), alpha=0.1, random_state=911))
models.append(MLPRegressor(hidden_layer_sizes=(250,5), alpha=0.1, random_state=911))
models.append(MLPRegressor(hidden_layer_sizes=(100,1), alpha=1.5, random_state=911))
models.append(MLPRegressor(hidden_layer_sizes=(250,5), alpha=1.5, random_state=911))

models.append(Lasso(alpha=0))
models.append(Lasso(alpha=0.05))
models.append(Lasso(alpha=0.1))
models.append(Lasso(alpha=0.2))
models.append(Lasso(alpha=0.5))
models.append(Lasso(alpha=1))
models.append(Lasso(alpha=2))
models.append(Lasso(alpha=5))

In [7]:
####Super learner
ensemble = SuperLearner(scorer=losscore2, random_state=911, verbose=2, folds=10,n_jobs=-1)
# Build the first layer XGBoost
ensemble.add(models)
# Attach the final meta estimator
ensemble.add_meta(LinearRegression())

SuperLearner(array_check=None, backend=None, folds=10,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=7858, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=10, raise_on_e...r=False))],
   n_jobs=-1, name='group-3', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=911, sample_size=20,
       scorer=make_scorer(losscore, greater_is_better=False),
       shuffle=False, verbose=2)

In [8]:
#Try
xt=np.array(x_train)
yt=np.array(y_train)

In [9]:
ensemble.fit(xt,yt)
preds = ensemble.predict(xt)


Fitting 2 layers
Processing layer-1             

c:\Users\hp\anaconda3\lib\site-packages\mlens\parallel\learner.py:179: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self.estimator.fit(xtemp, ytemp)
c:\Users\hp\anaconda3\lib\site-packages\mlens\parallel\learner.py:179: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self.estimator.fit(xtemp, ytemp)
c:\Users\hp\anaconda3\lib\site-packages\mlens\parallel\learner.py:179: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self.estimator.fit(xtemp, ytemp)
c:\Users\hp\anaconda3\lib\site-packages\mlens\parallel\learner.py:179: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  self.estimator.fit(xtemp, ytemp)
c:\Users\hp\anaconda3\lib\site-packages\mlens\parallel\learner.py:179: UserWarning: With alpha=0

done | 01:39:48
Processing layer-2             

c:\Users\hp\anaconda3\lib\site-packages\mlens\parallel\_base_functions.py:225: MetricWarning: [linearregression.0.0] Could not score linearregression. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  warnings.warn("[%s] Could not score %s. Details:\n%r" %


done | 00:00:00
Fit complete                        | 01:39:52

Predicting 2 layers
Processing layer-1             done | 00:00:02
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:03


In [11]:
predsdf=pd.DataFrame(preds)
predsdf.to_csv("y_preds_train.csv",index=False)

In [12]:
xtt=np.array(x_test)
preds_cali = ensemble.predict(xtt)


Predicting 2 layers
Processing layer-1             done | 00:00:00
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:03


In [13]:
preds_calidf=pd.DataFrame(preds_cali)
preds_calidf.to_csv("y_preds_test.csv",index=False)

In [14]:
#Desestandarizar
ytrainpred=preds*541552700.6981763+744197549.3413885
ytestpred=preds_cali*601842533+555314430


In [15]:
#Set directory:
os.chdir("C:/Users/hp/OneDrive - Universidad de los Andes/Documentos/Docs/Universidad/2022-2/Big Data/Taller 3/Repo/PS3_BD-ML")
tr=pyr.read_r("train.Rds") 
tt=pyr.read_r("test.Rds")
print(tr.keys())
print(tt.keys())
df_tr=tr[None] #Train Data frame (Bogotá D.C. y Medellín). 
df_tt=tt[None] #Test Data frame (Cali).

odict_keys([None])
odict_keys([None])


In [16]:
df_tr["price"]=df_tr["price"].astype(float)
price=df_tr["price"].tolist()

###Priors
percentile=[stats.percentileofscore(price,i,kind='weak') for i in price]
df_tr["percentil"]=pd.DataFrame(percentile)
df_tr["decil"]=pd.cut(df_tr["percentil"], bins=[0,10,20,30,40,50,60,70,80,90,100],labels=[0,1,2,3,4,5,6,7,8,9])
df_tr["percentil"]=pd.DataFrame(percentile)
df_tr["decil"]=pd.cut(df_tr["percentil"], bins=[0,10,20,30,40,50,60,70,80,90,100],labels=[0,1,2,3,4,5,6,7,8,9])

##cortes deciles
qtr=df_tr["price"].quantile([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]).reset_index()
qtr["index"]=qtr["index"]*10
diccortes=dict(zip(qtr.index, qtr.price))
df_tr["decil2"]=df_tr["decil"]
df_tr["cdecil"]=pd.DataFrame(df_tr["decil2"]).replace({'decil2':diccortes})

In [22]:
priortr=df_tr["cdecil"]

ytrainpred2=pd.DataFrame(ytrainpred)
priortr2=pd.DataFrame(priortr)
ytestpred2=pd.DataFrame(ytestpred)

In [23]:
##Prior train
y_prior=pd.merge(ytrainpred2,priortr2,left_index=True,right_index=True)
y_prior2=np.array(y_prior)
y_trainprior=np.array(df_tr["price"])

price=ytestpred2[0].tolist()
percentil=[stats.percentileofscore(price,i,kind='weak') for i in price]
decil=pd.cut(percentil, bins=[0,10,20,30,40,50,60,70,80,90,100],labels=[0,1,2,3,4,5,6,7,8,9])

In [24]:
import random
random.seed(911)
from random import choices

In [25]:

bog=df_tr[df_tr["city"]=="Bogotá D.C"]
med=df_tr[df_tr["city"]=="Medellín"]
bogprice=(bog["price"]-236337171)*1.07
bogprice=pd.DataFrame(bogprice)
medprice=(med["price"]-202000000)*1.36
medprice=pd.DataFrame(medprice)
pricesbogmed=bogprice.append(medprice)


C:\Users\hp\AppData\Local\Temp\ipykernel_10892\2067894571.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pricesbogmed=bogprice.append(medprice)


In [44]:
pricesbogmed

,price
0,5.817192e+08
1,9.241192e+08
2,1.084619e+09
3,5.496192e+08
4,6.565666e+08
...,...
51432,1.672800e+08
51433,3.073600e+08
51434,1.132880e+09
51435,3.780800e+08


In [45]:
pricesbogmed2=np.array(pricesbogmed["price"])
pricesbogmed2

array([5.81719227e+08, 9.24119227e+08, 1.08461923e+09, ...,
       1.13288000e+09, 3.78080000e+08, 3.80800000e+07])

In [50]:
distrcali=np.random.choice(pricesbogmed2, size=10000)
distrcali=pd.DataFrame(distrcali)
qtt=distrcali.quantile([0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]).reset_index()

diccortes=dict(zip(qtt.index, qtt[0]))
decil=pd.DataFrame(decil).replace({0:diccortes})


In [51]:
diccortes
decil

,0
0,5.004800e+08
1,9.776192e+08
2,2.692800e+08
3,1.876800e+08
4,3.677192e+08
...,...
4995,3.677192e+08
4996,6.780192e+08
4997,5.004800e+08
4998,2.692800e+08


In [54]:
##Prior test
y_priortest=pd.merge(ytestpred2,decil,left_index=True,right_index=True)
y_priortest2=np.array(y_priortest)

In [55]:
def losscorep(y,ypred):
    penalty=np.abs(y-ypred)
    penalty=np.where(penalty<40000000, np.exp(penalty),penalty)
    penalty=np.where(penalty>0,np.exp(penalty),penalty)
    abs_exp=np.mean(penalty)
    return abs_exp
    
losscore2p=make_scorer(losscorep, greater_is_better=False)

In [56]:
y_prior2

array([[7.09083712e+08, 6.60000000e+08],
       [1.11622131e+09, 1.08500000e+09],
       [1.29527629e+09, 1.08500000e+09],
       ...,
       [1.10879949e+09, 8.15000000e+08],
       [4.77353344e+08, 4.60000000e+08],
       [2.43090368e+08, 2.00000000e+08]])

In [64]:
y_trainprior

array([7.800e+08, 1.100e+09, 1.250e+09, ..., 1.035e+09, 4.800e+08,
       2.300e+08])

In [60]:
ensembleprior = SuperLearner(scorer=losscore2p, random_state=911, verbose=2, folds=10)
# Build the first layer XGBoost
ensembleprior.add_meta(LinearRegression())
ensembleprior.fit(y_prior2,y_trainprior)
predicttrainprior = ensembleprior.predict(y_prior2)


Fitting 1 layers
Processing layer-1             done | 00:00:00


c:\Users\hp\anaconda3\lib\site-packages\mlens\parallel\_base_functions.py:225: MetricWarning: [linearregression.0.0] Could not score linearregression. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  warnings.warn("[%s] Could not score %s. Details:\n%r" %


Fit complete                        | 00:00:01

Predicting 1 layers
Processing layer-1             done | 00:00:00
Predict complete                    | 00:00:01


In [61]:
predicttrainprior

array([7.1097018e+08, 1.1126945e+09, 1.5108876e+09, ..., 1.4161823e+09,
       4.3266618e+08, 2.1983952e+08], dtype=float32)

In [68]:
np.mean(np.abs(y_trainprior-predicttrainprior))

151504700.02656162

In [72]:
np.sqrt(np.mean((y_trainprior-predicttrainprior)**2))

420457478.76266

In [80]:
np.log(np.mean((np.abs(y_trainprior-predicttrainprior)**2.718)))

55.81849058940813

In [81]:
np.log(np.mean((np.abs(y_trainprior-ytrainpred)**2.718)))

56.14511240166749

In [73]:
losscorep(y_trainprior,predicttrainprior)

C:\Users\hp\AppData\Local\Temp\ipykernel_10892\1726069063.py:3: RuntimeWarning: overflow encountered in exp
  penalty=np.where(penalty<40000000, np.exp(penalty),penalty)
C:\Users\hp\AppData\Local\Temp\ipykernel_10892\1726069063.py:4: RuntimeWarning: overflow encountered in exp
  penalty=np.where(penalty>0,np.exp(penalty),penalty)


inf

In [62]:
predpriordf=pd.DataFrame(predicttrainprior)
predpriordf.to_csv("y_predsprior_train.csv",index=False)

In [63]:
predicttestprior = ensembleprior.predict(y_priortest2)
predpriordf_cali=pd.DataFrame(predicttestprior)
predpriordf_cali.to_csv("y_predsprior_test.csv",index=False)


Predicting 1 layers
Processing layer-1             done | 00:00:00
Predict complete                    | 00:00:01
